# Лабораторная работа №3
## Выполнил студент группы БВТ2003 Митрофанов Алексей Олегович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [118]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [119]:
def prefix_function(string: str) -> list[int]:
    prefix_lenghts = [0] * len(string)

    j = 0
    i = 1
    while i < len(string) and j < len(string):
        if string[i] == string[j]:
            j += 1
            prefix_lenghts[i] = j
            i += 1
        else:
            if j != 0:
                j = prefix_lenghts[j - 1]
            else:
                prefix_lenghts[i] = 0
                i += 1

    return prefix_lenghts


def kmp_find_pattern(_text: str, _pattern: str, case_sensitive = True) -> set[int]:
    # Applying case insensitivity in case 
    if not case_sensitive:
        text = _text.lower()
        pattern = _pattern.lower()
    else: 
        text = _text
        pattern = _pattern

    # Getting prefixes lengths for pattern
    pattern_prefixes = prefix_function(pattern)

    text_prefixes = [0] * len(text)
    occurences_indexes = set()

    i = 0
    j = 0
    while i < len(text):
        if text[i] == pattern[j]:
            j += 1
            text_prefixes[i] = j
            if j == len(pattern):
                occurences_indexes.add(i)
            j %= len(pattern)
            i += 1
        else:
            if j != 0:
                j = pattern_prefixes[j - 1]
            else:
                text_prefixes[i] = 0
                i += 1
    return occurences_indexes

#### Упрощенный алгоритм Бойера-Мура

In [120]:
def get_offset_table(string: str) -> dict[str:int]:
    offsets = {}
    table = [0] * len(string)

    for i in range(len(string) - 2, -1, -1):
        char = string[i]
        offset = len(string) - i - 1
        if char in offsets.keys():
            table[i] = offsets[char]
        else:
            offsets[char] = offset
            table[i] = offsets[char]
    if string[-1] in offsets.keys():
        table[-1] = offsets[string[-1]]
        return offsets
    else:
        offsets[string[-1]] = len(string)
        table[-1] = len(string)
        return offsets


def boyer_moore(_text: str, _pattern: str, case_sensitive = True) -> set[int]:
    if not case_sensitive:
        text = _text.lower()
        pattern = _pattern.lower()
    else:
        text = _text
        pattern = _pattern

    occurences_indexes = set()
    offset_table = get_offset_table(pattern)

    offset = 0
    index = len(pattern) - 1
    while index + offset < len(text):
        backwards = 0
        while backwards < len(pattern) and text[index + offset - backwards] == pattern[index - backwards]:
            backwards += 1
        if backwards == len(pattern):
            occurences_indexes.add(index+offset)
            offset += len(pattern)
        elif backwards > 0:
            offset += offset_table[pattern[-1]] 

        else:
            if  text[index+offset] in offset_table.keys():
                offset += offset_table[text[index+offset]]

            else:
                offset += len(pattern)
    return occurences_indexes

In [121]:

if input("Ручной ввод? y/n") == 'y': 
    text = input("Исходный текст: ")
    pattern = input("Образец поиска: ")
else:
    text = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
    pattern = "baaaaaaaaaaa"

if input("Чувствительный к регистру? y/n") == 'y':
    sensitivity = True
else:
    sensitivity = False

kmp = kmp_find_pattern(text, pattern, sensitivity)
b_m = boyer_moore(text, pattern, sensitivity)

print("Исходный текст: \n" + text)
print()
print("Образец для поиска: \n" + pattern)
print()
print("Найденные вхождения: ")
print("KMP: " + str(kmp))
print("B-M: " + str(b_m))

Исходный текст: 
aaaaaaaaaaaaaaaaaaaaaaaaaaaaa

Образец для поиска: 
baaaaaaaaaaa

Найденные вхождения: 
KMP: set()
B-M: set()


In [122]:
from time import perf_counter_ns as timer

results = {}

In [123]:
whole_time = 0
tries = 0
for _ in range(1000000):
    start = timer()
    kmp_find_pattern(text, pattern)
    end = timer()
    tries += 1
    whole_time += end - start
avg_time = whole_time / tries
results.update(dict(КМП=avg_time))
print(f"Time: {avg_time}")

Time: 8045.5007


In [124]:
whole_time = 0
tries = 0
for _ in range(1000000):
    start = timer()
    boyer_moore(text, pattern)
    end = timer()
    tries += 1
    whole_time += end - start
avg_time = whole_time / tries
results.update(dict(Б_М=avg_time))
print(f"Time: {avg_time}")

Time: 38080.6267


In [125]:
import re
whole_time = 0
tries = 0
for _ in range(1000000):
    start = timer()
    {m.start() for m in re.finditer(pattern, text)}
    end = timer()
    tries += 1
    whole_time += end - start
avg_time = whole_time / tries
results.update(dict(Встр=avg_time))
print(f"Time: {avg_time}")

Time: 1436.1855


In [126]:
leaderBoard = {k: v for k, v in sorted(results.items(), key=lambda item: item[1])}
for i, j in leaderBoard.items():
    print(f"{i}: \t{j} ms")

Встр: 	1436.1855 ms
КМП: 	8045.5007 ms
Б_М: 	38080.6267 ms


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [1]:
def check_solvability(tags: list[int]) -> bool:
    check_sum = 0
    for ind1, pivot in enumerate(tags):
        for ind2, element in enumerate(tags[ind1:]):
            if element < pivot and element != 0:
                check_sum += 1
    check_sum += tags.index(0) // 4 + 1

    return check_sum % 2 == 0

In [2]:
def possible_moves(position):
    n = 4
    blank = position.index(0)
    i, j = divmod(blank, n)
    offsets = []
    if i > 0:
        offsets.append(-n)
    if i < n - 1:
        offsets.append(n)
    if j > 0:
        offsets.append(-1)
    if j < n - 1:
        offsets.append(1)
    for offset in offsets:
        swap = blank + offset
        yield tuple(
            position[swap]
            if x == blank
            else position[blank]
            if x == swap
            else position[x]
            for x in range(n * n)
        )

In [3]:
def manhatan(state):
    solved = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
    dist = 0
    for ind, (real, ideal) in enumerate(zip(state, solved)):
        if real != ideal:
            horizontal = abs(ind  % 4 - solved.index(real) % 4)
            vertical   = abs(ind // 4 - solved.index(real) // 4)
            dist      += horizontal + vertical
    return dist

In [4]:
class Node:
    def __init__(self, position, distance_from_start, previous_node):
        self.position            = position
        self.distance_from_start = distance_from_start
        self.previous_node       = previous_node

    def __lt__(self, other):
        return self.distance_from_start < other.distance_from_start

    def __str__(self):
        n =  4
        return "\n".join(
            (n * "{:3}").format(*[i % (n * n) for i in self.position[i:]])
            for i in range(0, n * n, n)
        )

In [6]:
# fmt: off
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)

start      = [1,   2,  3,  4, 
              5,   6,  7,  8, 
              9,  10, 11, 12, 
              13, 14,  0, 15]

tags       = [12,  5,  8,  7,
               4, 11,  2, 13,
               6,  1,  0, 10,
               9, 15, 14,  3]

unsolvable = [1,   2,  3,  4, 
              5,   6,  7,  8, 
              9,  10, 11, 12, 
              13, 15,  14, 0]

# fmt: on

# start = tags
# start = unsolvable_tags

In [7]:
from queue import PriorityQueue

if check_solvability(start) == 0:
    print("Can't solved")
else:
    start = tuple(start)

    current_state = Node(start, 0, None)

    candidates = PriorityQueue()
    candidates.put((0, current_state))

    visited = set([current_state])
    came_from = {current_state.position: None}

    while not candidates.empty() and current_state.position != SOLVED:
        cost, current_state = candidates.get()
        for position in possible_moves(current_state.position):
            if position not in visited:
                new_candidate = (
                    manhatan(position),
                    Node(
                        position, current_state.distance_from_start + 1, current_state
                    ),
                )
                candidates.put(new_candidate)
                came_from[position] = current_state
                visited.add(position)

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14  0 15

[15]

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15  0

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14  0 15



### Вывод